<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# NOTE!
#
# The following lines are not needed in Google Colab

#import piplite
#await piplite.install(['folium'])
#await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd
from enum import Enum

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
# Import folium Popup
from folium.vector_layers import Popup

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
# NOTE!!!
#
# The following lines are not needed in Google Colab.
# Use the next 2 cells instead to download the file and prep it for this lab

# Download and read the `spacex_launch_geo.csv`
#from js import fetch
#import io
#
#URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
#resp = await fetch(URL)
#spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
#spacex_df=pd.read_csv(spacex_csv_file)

In [4]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv

--2023-10-28 02:41:04--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7710 (7.5K) [text/csv]
Saving to: ‘spacex_launch_geo.csv’

spacex_launch_geo.c 100%[===================>]   7.53K  --.-KB/s    in 0s      

2023-10-28 02:41:04 (1.38 GB/s) - ‘spacex_launch_geo.csv’ saved [7710/7710]



In [5]:
spacex_df=pd.read_csv("/content/spacex_launch_geo.csv")
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Function that returns a folium object containing the CircleMarker objects for each launch site
def launch_locations():
  # Instantiate a feature group for the dataset
  launch_locs = folium.map.FeatureGroup()

  # For each launch site, add a CircleMarker object based on its coordinate (Lat, Long)
  # values as well as a pop up label with the name of the launch site.
  for idx, row in launch_sites_df.iterrows():
      launch_locs.add_child(
          folium.vector_layers.CircleMarker(
              [row['Lat'], row['Long']],
              radius=5, # define how big you want the circle markers to be
              color='yellow',
              fill=True,
              fill_color='blue',
              fill_opacity=0.6,
              popup=row['Launch Site']
              #tooltip=row['Launch Site']
          )
      )
  return launch_locs


In [10]:
# Initialize a new map centered around NASA
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Get launch locations
launch_locs = launch_locations()
# Add it to the map
site_map.add_child(launch_locs)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
# See the tail of the dataframe
spacex_df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [12]:
# Get counts of successes by launch site
spacex_df['Launch Site'].value_counts()

CCAFS LC-40     26
KSC LC-39A      13
VAFB SLC-4E     10
CCAFS SLC-40     7
Name: Launch Site, dtype: int64

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
# marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [14]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

success_df = spacex_df.copy()
success_df['color'] = success_df.apply(lambda row: 'green' if row['class'] else 'red', axis=1)
success_df.tail()

,Launch Site,Lat,Long,class,color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [15]:
# Check the ouput
success_df.tail()

,Launch Site,Lat,Long,class,color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [16]:
# Function to build marker clusters of launch success/failures for all launch sites
def build_success_markers():
  marker_cluster = MarkerCluster()
  for index, record in success_df.iterrows():
    marker = folium.Marker([record['Lat'], record['Long']],
                          icon=folium.Icon(color='white', icon_color=record['color'])
                          )
    marker_cluster.add_child(marker)

  return marker_cluster

In [17]:
# Build list of success markers for each launch site
marker_clusters = build_success_markers()

# Add to the existing map
site_map.add_child(marker_clusters)

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.



Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
# Initialize a new map centered around NASA
mouse_site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add launch locations
launch_locs = launch_locations()
mouse_site_map.add_child(launch_locs)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topleft',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add mouse position object to map
mouse_site_map.add_child(mouse_position)

# Display map
mouse_site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [20]:
# find coordinate of the closest coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Coordinates for launch site
# CCAFS_SLC_40 = (28.563197,	-80.576820)
# Coast = (28.56351, -80.56798)

# points = [[CCAFS_SLC_40[0], CCAFS_SLC_40[1]], [Coast[0], Coast[1]]]

# distance = calculate_distance(CCAFS_SLC_40[0], CCAFS_SLC_40[1], Coast[0], Coast[1])
# distance



In [21]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# distance_marker = folium.Marker(
#    [Coast[0], Coast[1]],
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# site_map.add_child(distance_marker)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [22]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=points, weight=1)
# site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

In [24]:
# Review this dataframe again
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [25]:
# Build individual frames for each location
CCAFS_LC_40 = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS LC-40']
CCAFS_SLC_40 = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40']
KSC_LC_39A = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'KSC LC-39A']
VAFB_SLC_4E = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'VAFB SLC-4E']

In [26]:
# Build a list of points of interest around CCAFS SLC-40
# CCAFS SLC-40 and CCAFS LC-40 are really close, so, we'll use the same list
CCAFS_points_of_interest = [('Highway', 28.56345, -80.57084),
                            ('Railway', 28.57212, -80.58526),
                            ('Residential area', 28.40046, -80.60497),
                            ('Airport', 28.51505, -80.79633),
                            ('Orlando', 28.5384, -81.3789),
                            ('Titusville', 28.6122, -80.8076),
                            ('US-1', 28.56673, -80.79826),
                            ('I-95', 28.56221, -80.85182),
                            ('Coast', 28.56351, -80.56798)]
CCAFS_points_of_interest

[('Highway', 28.56345, -80.57084),
 ('Railway', 28.57212, -80.58526),
 ('Residential area', 28.40046, -80.60497),
 ('Airport', 28.51505, -80.79633),
 ('Orlando', 28.5384, -81.3789),
 ('Titusville', 28.6122, -80.8076),
 ('US-1', 28.56673, -80.79826),
 ('I-95', 28.56221, -80.85182),
 ('Coast', 28.56351, -80.56798)]

In [35]:
# Build a list of points of interest around KSC LC-39A
KSC_points_of_interest = [('Highway', 28.57314, -80.65536),
                          ('Railway', 28.57367, -80.65415),
                          ('Residential area', 28.49857, -80.71587),
                          ('Airport', 28.51505, -80.79633),
                          ('Orlando', 28.5384, -81.3789),
                          ('Titusville', 28.6122, -80.8076),
                          ('US-1', 28.56673, -80.79826),
                          ('I-95', 28.56221, -80.85182),
                          ('Coast', 28.60592, -80.59708)]
KSC_points_of_interest

[('Highway', 28.57314, -80.65536),
 ('Railway', 28.57367, -80.65415),
 ('Residential area', 28.49857, -80.71587),
 ('Airport', 28.51505, -80.79633),
 ('Orlando', 28.5384, -81.3789),
 ('Titusville', 28.6122, -80.8076),
 ('US-1', 28.56673, -80.79826),
 ('I-95', 28.56221, -80.85182),
 ('Coast', 28.60592, -80.59708)]

In [28]:
# Build a list of points of interest around VAFB SLC-4E
VAFB_points_of_interest = [('CA 1', 34.62855, -120.43247),
                          ('Railway', 34.63301, -120.62499),
                          ('Residential area', 34.63151, -120.4838),
                          ('Airport', 34.66583, -120.46749),
                          ('Los Angeles', 34.04356, -118.24585),
                          ('Lompoc', 34.63942, -120.45822),
                          ('US 101', 34.79576, -120.34149),
                          ('Coast', 34.63275, -120.62647)]

VAFB_points_of_interest

[('CA 1', 34.62855, -120.43247),
 ('Railway', 34.63301, -120.62499),
 ('Residential area', 34.63151, -120.4838),
 ('Airport', 34.66583, -120.46749),
 ('Los Angeles', 34.04356, -118.24585),
 ('Lompoc', 34.63942, -120.45822),
 ('US 101', 34.79576, -120.34149),
 ('Coast', 34.63275, -120.62647)]

In [29]:
# Function to add points of interest as well as lines from the launch site
# to the points of interest
def build_markers(launch_site, poi_list):
  # Make a copy
  temp_list = poi_list.copy()

  # Add equator as a point of interest
  temp_list.append(('Equator', 0, launch_site.iloc[0, 2]))

  # Convert list to a dataframe
  poi_df = pd.DataFrame(temp_list, columns=['marker', 'lat', 'long'])

  # Compute distances
  poi_df['distance'] = poi_df.apply(lambda row: calculate_distance(launch_site.iloc[0, 1], launch_site.iloc[0, 2], row['lat'], row['long']), axis=1)

  # Instantiate a feature group for the dataset
  markers = folium.map.FeatureGroup()

  # Loop through the dataframe and process each row
  for idx, row in poi_df.iterrows():
      markers.add_child(
          # Add a marker for the point of interest
          folium.Marker([row['lat'], row['long']],
                        icon=DivIcon(
                        icon_size=(20,20),
                        icon_anchor=(0,0),
                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{}\n{:10.2f} KM".format(row['marker'],row['distance']),
                        )))
      # Add a line from launch site to point of interest
      points = [[launch_site.iloc[0, 1], launch_site.iloc[0, 2]], [row['lat'], row['long']]]
      markers.add_child(folium.PolyLine(locations=points, weight=1))

  # Return the list of markers and lines we just created
  return markers

In [30]:
# Function to build a map from scratch with the following:
#   - launch locations
#   - succes/fail clusters
#   - points of interest from launch site
def build_map(launch_site, points_of_interest):
  # Start with a clean map centered around the launch site and zoom in
  site_map = folium.Map(location=[launch_site.iloc[0, 1], launch_site.iloc[0,2]], zoom_start=15)

  # Add all launch locations
  launch_locs = launch_locations()
  site_map.add_child(launch_locs)

  # Add all success/fail clusters
  success_clusters = build_success_markers()
  site_map.add_child(success_clusters)

  # Add points of interest centered around a launch site
  poi_markers = build_markers(launch_site, points_of_interest)
  site_map.add_child(poi_markers)

  # Return the map
  return site_map

In [31]:
# Build map for CCAFS SLC-40
site_map = build_map(CCAFS_SLC_40, CCAFS_points_of_interest)
site_map

In [32]:
# Build map for CCAFS LC-40
site_map = build_map(CCAFS_LC_40, CCAFS_points_of_interest)
site_map

In [36]:
# Build map for KSC LC-39A
site_map = build_map(KSC_LC_39A, KSC_points_of_interest)
site_map

In [34]:
# Build map for VAFB SLC-4E
site_map = build_map(VAFB_SLC_4E, VAFB_points_of_interest)
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
